## ALS Implicit Collaborative Filtering - pos ratings

https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe

In [1]:
import import_ipynb

In [2]:
from evaluation import DCG
from evaluation import nDCG
from evaluation import R_Precision
from time import time

importing Jupyter notebook from evaluation.ipynb
DCG = 0.5
IDCG = 1.0
nDCG = 0.5


In [3]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random
import implicit

from sklearn.preprocessing import MinMaxScaler
from scipy.sparse.linalg import spsolve

# Recomendation and evaluation functions

In [4]:
#---------------------
# FIND SIMILAR ITEMS
#---------------------

def similar_items(seed_track, top_n):
    """
    input: track_uri
    output: top_n recommended track_uris
    """
    track_id = D_track_id[seed_track] 
    n_similar =  top_n

    # Use implicit to get similar items.
    similar = model.similar_items(track_id, n_similar)
    
    similar_i = []
    
    # Print the names of our most similar artists
    for item in similar:
        idx, score = item
        track_uri = data.track_uri.loc[data.track_uri_id == idx].iloc[0]
        #print(data.track_uri.loc[data.track_uri_id == idx].iloc[0], D_desc[track_uri])
        similar_i.append(data.track_uri.loc[data.track_uri_id == idx].iloc[0])
    return similar_i

In [35]:
#-------------------------------------
# FIND SIMILAR ITEMS WITH DESCRIPTION
#-------------------------------------

def similar_items_with_description(seed_track, top_n):
    """
    input: track_uri
    output: top_n recommended track_uris with description as dictionary
    """
    print('CF ALS pos - first track returned is the seed track')
    
    track_id = D_track_id[seed_track] 
    n_similar =  top_n+1

    # Use implicit to get similar items.
    similar = model.similar_items(track_id, n_similar)
    
    similar_i = {}
    
    # Print the names of our most similar artists
    for item in similar:
        idx, score = item
        track_uri = data.track_uri.loc[data.track_uri_id == idx].iloc[0]
        similar_i[track_uri] = D_desc[track_uri]
        #print(data.track_uri.loc[data.track_uri_id == idx].iloc[0], D_desc[track_uri])
        #similar_i.append(data.track_uri.loc[data.track_uri_id == idx].iloc[0])
    return list(similar_i.values())

In [6]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

def create_recs(pid,N):
    """
    returna list
    """
    
    pid_id = pid

    # Use the implicit recommender.
    recommended = model.recommend(pid_id, sparse_user_item, N=N)

    tracks = []
    scores = []
    desc = []

    # Get artist names from ids
    for item in recommended:
        idx, score = item
        tracks.append(data.track_uri.loc[data.track_uri_id == idx].iloc[0])
        scores.append(score)
        #desc.append(D_desc[data.track_uri.loc[data.track_uri_id == idx].iloc[0]])

    # Create a dataframe of artist names and scores
    #recommendations = pd.DataFrame({'track_uris': tracks, 'score': scores})

    return tracks

In [68]:
#----------------------------------------------
# CREATE USER RECOMMENDATIONS WITH DESCRIPTION
#----------------------------------------------

def create_recs_with_description(pid,N):
    pid_id = pid

    # Use the implicit recommender.
    I = {}
    for el in data[data.pid == pid_id].track_uri.unique():
        I[el] = D_desc[el]
    
    recommended = model.recommend(pid_id, sparse_user_item, N=N)
    
    R = {}

    for item in recommended:
        idx, score = item
        R[data.track_uri.loc[data.track_uri_id == idx].iloc[0]] = D_desc[data.track_uri.loc[data.track_uri_id == idx].iloc[0]]
    
    return list(I.values()), list(R.values())

In [8]:
#----------------------------------------------------
# CREATE USER RECOMMENDATIONS WITH DICTIONARY OUTPUT
#---------------------------------------------------
def create_recs_dictionary_output(pid,N):
    """
    input: pid
    output: reccomendation dictionary {track_uri: score}
    """
    
    pid_id = pid

    # Use the implicit recommender.
    recommended = model.recommend(pid_id, sparse_user_item, N=N)

    rec_tracks = {}

    # Get artist names from ids
    for item in recommended:
        idx, score = item
        rec_tracks[D_track_id_to_uri[idx]] = score
#         tracks.append(data.track_uri.loc[data.track_uri_id == idx].iloc[0])
#         scores.append(score)
        #desc.append(D_desc[data.track_uri.loc[data.track_uri_id == idx].iloc[0]])

    # Create a dataframe of artist names and scores
    #recommendations = pd.DataFrame({'track_uris': tracks, 'score': scores})

    return rec_tracks

In [9]:
#----------------------------------
# GET RECOMMENDATIONS AND EVALUATE
#----------------------------------

def als_predict_and_evaluate_top_n(pid, top_n=100):
    """
    return
    (1) top_n predicted: track_ids
    (2) ground_truth : track_ids in the hold_out
    (3) R_Prec
    (4) NDGC
    
    """
    L_pred = list(create_recs_dictionary_output(pid,top_n).keys())
    
    ground_truth = ev_set_arr[ev_set_arr[:,0]==pid][:,1]
    
    R_Prec = R_Precision(L_pred[:len(ground_truth)],ground_truth)
    
    res = [int(el in ground_truth) for el in L_pred]
    
    NDCG = nDCG(res)[1]
    
    return L_pred, ground_truth, R_Prec, NDCG, res

In [10]:
#-----------------------------------
# SAVE R-PRECISION AND NDCG BY PID
#-----------------------------------

def save_als_res_k_n(n = 10, top_n=20):
    """
    k = number of factors
    n= number of random lists to predict
    """
    time0=time()
    RES={}
    ep = random.sample(evaluation_pids,n)
    for i,pid in enumerate(ep):
        predictions=als_predict_and_evaluate_top_n(pid,top_n)
        RES[pid] = [predictions[2], predictions[3]]
        if i % 500 ==0:
            print(i)
            print(time()-time0)
    df = pd.DataFrame(RES).transpose().reset_index()
    df.columns=['pid','R-Precision','nDCG']
    df['rating'] = 'pos'
    df['model'] = f'ALS'
    df.to_csv(f'../evaluation/ALS_pos_topn_{top_n}_{n}.csv', index = None)
    print(time()-time0)
    return df

# Load data

In [11]:
file_path = '../data-processed/full-data/pid-track-pos-rating-train-data.csv'

In [12]:
# Load the data like we did before
raw_data = pd.read_csv(file_path)
# raw_data.head()

,pid,track_uri,pos_rating
0,491000,spotify:track:3giQ7393501IRNrd8iHugf,10.000000
1,491000,spotify:track:3jpcVaeyNjWgjqIxAiWasz,9.949153
2,491000,spotify:track:1uuqRaSJAiQ6VB8BWblXWJ,9.898305
3,491000,spotify:track:7gXpcXwtmEiQzskYJmtGgk,9.847458
4,491000,spotify:track:5wtIWwOtowY2howCZ7Veq2,9.796610


In [13]:
raw_data.columns = ['pid', 'track_uri', 'rating']
data = raw_data

In [14]:
# Create a numeric user_id and artist_id column
data['pid'] = data['pid'].astype("category")
data['track_uri'] = data['track_uri'].astype("category")
data['pid_id'] = data['pid'].cat.codes
data['track_uri_id'] = data['track_uri'].cat.codes
# data.head()

,pid,track_uri,rating,pid_id,track_uri_id
0,491000,spotify:track:3giQ7393501IRNrd8iHugf,10.000000,491000,947381
1,491000,spotify:track:3jpcVaeyNjWgjqIxAiWasz,9.949153,491000,960223
2,491000,spotify:track:1uuqRaSJAiQ6VB8BWblXWJ,9.898305,491000,493113
3,491000,spotify:track:7gXpcXwtmEiQzskYJmtGgk,9.847458,491000,1916386
4,491000,spotify:track:5wtIWwOtowY2howCZ7Veq2,9.796610,491000,1526083


In [15]:
D_track_id = data.groupby('track_uri')['track_uri_id'].min().to_dict()

In [16]:
D_track_id_to_uri = {}
for k,v in D_track_id.items():
    D_track_id_to_uri[v] = k

# Build ALS model

In [17]:
# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['rating'].astype(float), (data['track_uri_id'], data['pid_id'])))
sparse_user_item = sparse.csr_matrix((data['rating'].astype(float), (data['pid_id'], data['track_uri_id'])))

In [18]:
# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

In [19]:
# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

In [20]:
#Fit the model
model.fit(data_conf)

# Load dictionary with tracks

In [21]:
import json

with open('../data-processed/full-data/track_descriptions.json') as json_file:
    D_desc = json.load(json_file)
    
D_desc['spotify:track:0UaMYEvWZi0ZqiDOoHU3YI']

['Lose Control (feat. Ciara & Fat Man Scoop)', 'Missy Elliott', 'The Cookbook']

# Find similar tracks

 <img src="tenor.gif">

In [36]:
similar_items_with_description('spotify:track:5fwP61Nmo5xXYIjO9XEWqb',15)

CF ALS pos - first track returned is the seed track


[["I'm Still Standing", 'Elton John', 'Too Low For Zero'],
 ['The Power Of Love', 'Huey Lewis & The News', 'Collected'],
 ['Maneater - Remastered', 'Daryl Hall & John Oates', 'H2O'],
 ['Karma Chameleon - 2002 - Remaster', 'Culture Club', 'Colour By Numbers'],
 ['Video Killed The Radio Star - Single Version',
  'The Buggles',
  'Video Killed The Radio Star / Kid Dynamo'],
 ['Walk Like an Egyptian', 'The Bangles', 'Different Light'],
 ["Don't You Forget About Me", 'Simple Minds', "Don't You Forget About Me"],
 ['Holding Out for a Hero',
  'Bonnie Tyler',
  'Secret Dreams And Forbidden Fire'],
 ['Video Killed The Radio Star', 'The Buggles', 'The Age Of Plastic'],
 ['Break My Stride', 'Matthew Wilder', "I Don't Speak The Language"],
 ['Dancing With Myself', 'Billy Idol', 'VH1 Storytellers'],
 ['Our House', 'Madness', 'The Rise & Fall'],
 ['99 Luftballons', 'Nena', '2 in 1 Nena'],
 ["You're the Voice", 'John Farnham', 'Whispering Jack'],
 ['The Safety Dance', 'Men Without Hats', 'Rhythm of 

# Create a playlist continuation 

In [74]:
print('Input')
inp = 491004
create_recs_with_description(inp, 20)[0][:10]

Input


[['Ignition - Remix', 'R. Kelly', 'Chocolate Factory'],
 ['Single Ladies (Put a Ring on It)',
  'Beyoncé',
  'I AM...SASHA FIERCE - Platinum Edition'],
 ['Untitled (How Does It Feel)', "D'Angelo", 'The Best So Far'],
 ['U Got It Bad', 'Usher', '8701'],
 ["You Don't Know My Name", 'Alicia Keys', 'The Diary Of Alicia Keys'],
 ['Love Of My Life (An Ode To Hip Hop) - Longer Album Version',
  'Erykah Badu',
  'Go! Common Classics'],
 ['Be Without You', 'Mary J. Blige', 'The Breakthrough'],
 ['Thrift Shop (feat. Wanz)',
  'Macklemore & Ryan Lewis',
  'The Heist [Deluxe Edition]'],
 ['No Scrubs', 'TLC', 'Fanmail'],
 ['Regulate', 'Warren G', 'Regulate… G Funk Era']]

In [75]:
print('output')
create_recs_with_description(inp, 20)[1]

output


[['Pony', 'Ginuwine', 'R&B: From Doo-Wop To Hip-Hop'],
 ['No Diggity', 'Blackstreet', 'Another Level'],
 ['Dilemma', 'Nelly', 'Nellyville'],
 ["Bump n' Grind", 'R. Kelly', '12 Play'],
 ['Say My Name', "Destiny's Child", "The Writing's On The Wall"],
 ['My Boo', 'Usher', 'Confessions'],
 ['Let Me Love You', 'Mario', 'Turning Point'],
 ['Crazy In Love', 'Beyoncé', 'Dangerously In Love (Alben für die Ewigkeit)'],
 ['Always On Time', 'Ja Rule', 'Pain Is Love'],
 ['Yeah!', 'Usher', 'Confessions'],
 ['Suga Suga', 'Baby Bash', "Tha Smokin' Nephew"],
 ['My Love', 'Justin Timberlake', 'FutureSex/LoveSounds'],
 ['Rock Your Body', 'Justin Timberlake', 'Justified'],
 ['This Is How We Do It', 'Montell Jordan', 'This Is How We Do It'],
 ['You Make Me Wanna...', 'Usher', 'My Way'],
 ['Miss Independent', 'Ne-Yo', 'Year Of The Gentleman'],
 ['Let Me Blow Ya Mind', 'Eve', 'Scorpion'],
 ['Ride Wit Me', 'Nelly', 'Country Grammar'],
 ['Killing Me Softly with His Song', 'Fugees', 'The Score'],
 ["What's Luv

# Evaluation

In [25]:
evaluation_set = pd.read_csv('../data-processed/full-data/evaluation-pids-ground-truth.csv')
evaluation_set.head()

,pid,pos,track_uri,hold_out,seed_pattern
0,491004,0,spotify:track:5dNfHmqgr128gMY2tc5CeJ,0,first n
1,491004,1,spotify:track:557un1HgwYMuqfWGSTmnxw,0,first n
2,491004,2,spotify:track:6Ms01Gqi8gVBs14YrNUlVZ,0,first n
3,491004,3,spotify:track:7J41dYQolQJEtj3UmKLu5r,0,first n
4,491004,4,spotify:track:6LGwYMXXgURfaequXipzHx,0,first n


In [26]:
ev_set = evaluation_set[evaluation_set['hold_out'] == 1][['pid','track_uri','hold_out']]
ev_set = ev_set[ev_set.isnull()==False]

ev_set_arr = ev_set.to_numpy()

evaluation_pids = list(ev_set.pid.unique())

# ev_set.head()

,pid,track_uri,hold_out
38,491004,spotify:track:7i7UIbm5E0DD7aSOYvwp2v,1
39,491004,spotify:track:0FZvjrHpAmLKj574M4VwrF,1
40,491004,spotify:track:0rE8OLQpoOFe2xa35twgve,1
41,491004,spotify:track:4rY3KONqdimczgu2NhxI8k,1
42,491004,spotify:track:6DkXLzBQT7cwXmTyzAB1DJ,1


### Save evaluation

In [27]:
# df = save_als_res_k_n(10000,500)
# df.describe()